# FORMULA 1 LENOVO CHINESE GRAND PRIX 2024

The Chinese Grand Prix (Chinese: 中国大奖赛; pinyin: Zhōngguó Dàjiǎngsài) is a round of the Formula One World Championship. The event was held every year from 2004 until 2019 and is contracted to be held until 2025.[1] The event was contracted to take place from 2020 to 2023, but was cancelled each of those years as a consequence of the COVID-19 pandemic in China.

It is currently held at the Shanghai International Circuit, Jiading, Shanghai. Designed by Hermann Tilke and opened in 2004, the US240 million Shanghai course was the most expensive Formula One circuit facility[2] until the $6 billion Abu Dhabi course opened five years later.[3] The 5.451 km (3.387 mi) track features one of the trickiest corner combinations on the Formula One calendar: Turn 1 and 2, a demanding 270-degree, right-handed corner combination whose radius decreases as the corner progresses. Source: Wikipedia

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

First of all it is neccesary to obtain all the data of the 2024 sessions in order to obtain some data as session_key,country_key,country_code and circuit_key

In [ ]:
response = urlopen('https://api.openf1.org/v1/sessions?year=2024&&country_code=CHN')
data = json.loads(response.read().decode('utf-8'))
calendar = pd.DataFrame(data)
calendar

## Setup
It will be saved those attributes aforementioned.

In [ ]:
country_key='country_key=53'
country_code="country_code='CHN'"
circuit_key='circuit_key=49'
year = 'year=2024'

## Free Practice 1

### Laps

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
session_key='session_key=9663'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
practice1 = pd.DataFrame(data)
practice1

### Stints

In [ ]:
#Stints
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

### Drivers

In [ ]:
# Name of the drivers
url = 'https://api.openf1.org/v1/drivers?'
# In this case, due to Open URL does not refresh well the feature drivers, I will catch Practice 1 drivers as a reference.
session_key = 'session_key=9472'
urltotal = url+session_key

response = urlopen(urltotal)
drivers = json.loads(response.read().decode('utf-8'))
drivers = pd.DataFrame(drivers)
# Restore the session key
session_key='session_key=9663'
drivers

### Merge tables

In [ ]:
stintsDataFrame = pd.DataFrame()
for index,row in stintInformation.iterrows():
    number_driver = row.driver_number
    acronym_driver = drivers.query('driver_number == @number_driver').name_acronym.to_string(index=False)
    full_name = drivers.query('driver_number == @number_driver').full_name.to_string(index=False)
    team_name = drivers.query('driver_number == @number_driver').team_name.to_string(index=False)
    if row.lap_start != row.lap_end:
        contador = row.lap_start
        while contador <=row.lap_end :
            new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':contador,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
            stintsDataFrame = pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)
            contador+=1
    else:
    
        new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':row.lap_start,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
        stintsDataFrame =pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
jointables = pd.merge(practice1,stintsDataFrame,on=['lap_number','driver_number'])
jointables

### Data Tyres

In [ ]:
def define_colour(compound):
    if compound == "SOFT":
        colour = "red"
    elif compound == "MEDIUM":
        colour = "yellow"
    elif compound == "HARD":
        colour = "grey"
    elif compound == "INTERMEDIATE":
        colour = "green"
    else:
        colour = "blue"
    return colour

In [ ]:
def show_plot(arrayDataframes,colour):
    figure, axis = plt.subplots(len(arrayDataframes),figsize=(15,85))
    #plt.xlim(92, 96)
    i=0
    for arr in arrayDataframes:
        arr.reset_index(drop=True, inplace=True)
        axis[i].plot( arr.lap_duration,marker ="o",color=colour,label = str(arr.full_name[0]) )
        axis[i].set_xlabel("Lap time")
        axis[i].set_ylabel("Lap Time Seconds")
        axis[i].legend()
        i = i+1

In [ ]:
def obtain_data_tyres(dataset,compound,race):
    extra = ''
    if race == False:
        extra = ' and lap_duration <112'
    else:
        extra = 'and lap_duration < 110'
    setTyres = dataset.query('compound == @compound '+extra)
    drivers_number = []
    set_dict = {}
    for index,row in setTyres.iterrows():
        if row.driver_number not in drivers_number:
            set_dict[row.driver_number] = []
            drivers_number.append(row.driver_number)

        set_dict[row.driver_number].append(row)

    arrayDataframes = []
    for valor in set_dict.values():
        arrayDataframes.append(pd.DataFrame(valor))
    colour = define_colour(compound)
    show_plot(arrayDataframes,colour)

In [ ]:
obtain_data_tyres(jointables,"SOFT",False)

In [ ]:
obtain_data_tyres(jointables,"MEDIUM",False)

In [ ]:
obtain_data_tyres(jointables,"HARD",False)

In [ ]:
# Source: https://www.geeksforgeeks.org/how-to-annotate-bars-in-barplot-with-matplotlib-in-python/
def obtainchart(xvariable,yvariable,dataset):
    plt.figure(figsize=(12, 9))
    plots = sns.barplot(x=xvariable, y=yvariable, data=dataset,color='yellow')
    for bar in plots.patches:
        plots.annotate(format(bar.get_height(), '.3f'), 
                       (bar.get_x() + bar.get_width() / 2, 
                        bar.get_height()), ha='center', va='center',
                       size=8, xytext=(0, 7),
                       textcoords='offset points')
    plt.show()

### Top speed

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

### Fastest lap per compound

In [ ]:
compoundsPace = jointables.loc[jointables.groupby(['compound'])['lap_duration'].idxmin()]
compoundsPace[['full_name','compound','duration_sector_1','duration_sector_2','duration_sector_3','lap_duration']]

### Fastest lap

In [ ]:
def obtain_fastest_lap(driver,dataset,newdataset):
    fastest_lap = dataset.query("driver_number == @driver").lap_duration.min()
    team_name = dataset.query("driver_number == @driver").head(1).team_name.to_string(index=False)
    #team_colour = dataset.query("driver_number == @driver").head(1).team_colour.to_string(index=False)
    name_acronym = dataset.query('driver_number == @driver').head(1).name_acronym.to_string(index=False)
    new_row = {'driver_number':driver,'fastest_lap':fastest_lap,'name_acronym': name_acronym, 'team_name':team_name}
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
def obtain_deltas(dataset,array):
    fastest_lap = dataset.fastest_lap.min()
    for row in dataset.iterrows():
        lap = row[1][1]
        delta = lap-fastest_lap
        array.append(delta)
    return array

In [ ]:
practiceCleaned = jointables.query("lap_duration >1")
drivers_list = list(practiceCleaned['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_fastest_lap(driver,practiceCleaned,newdataset)

In [ ]:
arr= obtain_deltas(newdataset,[])
newdataset.insert(3,'delta',arr)

In [ ]:
dt = newdataset.sort_values(ascending=True,by='delta')
obtainchart("name_acronym","delta",dt)

### Lap duration

In [ ]:
boxplot = jointables.query("is_pit_out_lap == False and lap_duration < 100").boxplot(column='lap_duration',by='name_acronym',fontsize=5)
boxplot.get_figure().suptitle('drivers')

### Track dominance

Note: It must not compare Fernando and Gasly laptimes because they did not used soft tyres.
### Sector 1
Both McLaren drivers were fastest on sector 1 with 3 tenths of difference beetween Lando and  Stroll.


In [ ]:
sectorPace = jointables.loc[jointables.groupby(['driver_number'])['duration_sector_1'].idxmin()].sort_values(by=['duration_sector_1'],ascending=True)
sectorPace[['duration_sector_1','full_name','compound','lap_duration','lap_number']]

### Sector 2
Lance Stroll was the fastest but in terms of team, Red Bull was the fastest with 4 hundreths between Max and Checo

In [ ]:
sectorPace = jointables.loc[jointables.groupby(['driver_number'])['duration_sector_2'].idxmin()].sort_values(by=['duration_sector_2'],ascending=True)
sectorPace[['duration_sector_2','full_name','compound','lap_duration','lap_number']]

### Sector 3
In this case Checo was the fastest followed by Max. A thing that we should take into account is the pace shown by both Red Bulls in this sector that could give an hint about what it could happen in qualyfing.

In [ ]:
sectorPace = jointables.loc[jointables.groupby(['driver_number'])['duration_sector_3'].idxmin()].sort_values(by=['duration_sector_3'],ascending=True)
sectorPace[['duration_sector_3','full_name','compound','lap_duration','lap_number']]

### Race pace per teams

In [ ]:
def getinfolonglaps(dataset,driver_number,team):
    dataset = dataset.query("is_pit_out_lap == False and driver_number == @driver_number and team_name == @team and lap_duration < 106 and lap_duration >94 ")
    return dataset[['full_name','compound','date_start','lap_number','duration_sector_1','duration_sector_2','duration_sector_3','lap_duration']]

#### Red Bull Racing

In [ ]:
stintInformation.query('driver_number == 1 or driver_number == 11')

In [ ]:
getinfolonglaps(jointables,1,'Red Bull Racing')

In [ ]:
getinfolonglaps(jointables,11,'Red Bull Racing')

#### Ferrari

In [ ]:
stintInformation.query('driver_number == 16 or driver_number == 55')

In [ ]:
getinfolonglaps(jointables,16,'Ferrari')

In [ ]:
getinfolonglaps(jointables,55,'Ferrari')

#### Mercedes

In [ ]:
stintInformation.query('driver_number == 63 or driver_number == 44')

In [ ]:
getinfolonglaps(jointables,63,'Mercedes')

In [ ]:
getinfolonglaps(jointables,44,'Mercedes')

#### Aston Martin

In [ ]:
stintInformation.query('driver_number == 14 or driver_number == 18')

In [ ]:
getinfolonglaps(jointables,14,'Aston Martin')

In [ ]:
getinfolonglaps(jointables,18,'Aston Martin')

#### McLaren

In [ ]:
stintInformation.query('driver_number == 4 or driver_number == 81')

In [ ]:
getinfolonglaps(jointables,81,'McLaren')

In [ ]:
getinfolonglaps(jointables,4,'McLaren')

#### RB

In [ ]:
stintInformation.query('driver_number == 3 or driver_number == 22')

In [ ]:
getinfolonglaps(jointables,3,'RB')

In [ ]:
getinfolonglaps(jointables,22,'RB')

#### Williams

In [ ]:
stintInformation.query('driver_number == 2 or driver_number == 23')

In [ ]:
getinfolonglaps(jointables,2,'Williams')

In [ ]:
getinfolonglaps(jointables,23,'Williams')

#### Kick Sauber

In [ ]:
stintInformation.query('driver_number == 24 or driver_number == 77')

In [ ]:
getinfolonglaps(jointables,24,'Kick Sauber')

In [ ]:
getinfolonglaps(jointables,77,'Kick Sauber')

#### Alpine

In [ ]:
stintInformation.query('driver_number == 10 or driver_number == 31')

In [ ]:
getinfolonglaps(jointables,10,'Alpine')

In [ ]:
getinfolonglaps(jointables,31,'Alpine')

#### Haas

In [ ]:
stintInformation.query('driver_number == 20 or driver_number == 27')

In [ ]:
getinfolonglaps(jointables,20,'Haas')

In [ ]:
getinfolonglaps(jointables,27,'Haas')

Seeing the charts and also the long stints I think that one stop it could not be an option because there is a lot of degradation and I think that among lap 13-18, drivers would have to change their tyres.They have to change among these lap because since lap 14 we can see a difference of 1 second beetween these sets of tyres. In case of hards tyres, degratation are low and it could be a good tyre in the medium stint. Among 38-43 lap they should stop to change for mediums.
Soft would not be used as a race tyre.

## Sprint Qualyfing

### Race control
This section has been added in order to know which laps has been deleted and knowing what happened on track during this session as well.

In [ ]:
url = 'https://api.openf1.org/v1/race_control?'
session_key = 'session_key=9668'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race_control = pd.DataFrame(data)
race_control.query("category== 'Other'")

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
qualyfing = pd.DataFrame(data)
qualyfing

In race control dataset, I can see a lot of laptimes deleted, principally for track limits. Those laps deleted were deleted from dataset in order to obtain only the valid laps for the analysis.

In [ ]:
qualyfing = qualyfing.drop(122)
qualyfing = qualyfing.drop(183)
qualyfing = qualyfing.drop(181)
qualyfing = qualyfing.drop(196)
qualyfing = qualyfing.drop(199)
qualyfing = qualyfing.drop(193)
qualyfing = qualyfing.drop(191)
qualyfing = qualyfing.drop(204)

As the qualyfing session has been with weather changes, I will take two fastests lap as a reference to obtain all the competitive laps. For this reason, I show two laps, one with dry conditions and other with wets.

In [ ]:
bestlap_dry = qualyfing.loc[qualyfing.groupby(['driver_number'])['lap_duration'].idxmin()].sort_values(by=['lap_duration'],ascending=True)
bestlap_dry[0:1]

In [ ]:
bestlap_wet = qualyfing.query("date_start>='2024-04-19T08:11:00'").loc[qualyfing.query("date_start>='2024-04-19T08:11:00'").groupby(['driver_number'])['lap_duration'].idxmin()].sort_values(by=['lap_duration'],ascending=True)
bestlap_wet[0:1]

In this case, the fastest lap in dry conditions is 95.606 seconds (1.35.606= so that to obtain the competitve laps the fastest lap will be multiplied by 1.07 (102.298 seconds) due to, according to the rules all the drivers have to do unless one lap within this gap.

In [ ]:
competitiveLaps_dry = qualyfing.query("is_pit_out_lap == False and lap_duration < 102.298")
competitiveLaps_dry

In this case, the fastest lap in wet conditions is 117.94 seconds (1.57.94= so that to obtain the competitve laps the fastest lap will be multiplied by 1.07 (126.195 seconds) due to, according to the rules all the drivers have to do unless one lap within this gap.

In [ ]:
competitiveLaps_wet = qualyfing.query("date_start>='2024-04-19T08:11:00' and is_pit_out_lap == False and lap_duration < 126.1958")
competitiveLaps_wet

In [ ]:
def obtain_information_qualy(driver,dataset,newdataset):
    fastest_lap = dataset.query("driver_number == @driver").lap_duration.min()
    fastest_lap_absolute = dataset.lap_duration.min()
    delta = fastest_lap - fastest_lap_absolute
    st_speed = dataset.query("driver_number == @driver").st_speed.min()
    i1_speed = dataset.query("driver_number == @driver").i1_speed.min()
    i2_speed = dataset.query("driver_number == @driver").i2_speed.min()
    new_row = {'driver_number':driver,'fastest_lap':fastest_lap,'delta': delta,'st_speed':st_speed,'i1_speed':i1_speed,'i2_speed':i2_speed}
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
drivers_list = list(competitiveLaps_dry['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_information_qualy(driver,competitiveLaps_dry,newdataset)
jointables = pd.merge(newdataset,drivers,on=['driver_number'])
jointables.sort_values(by=['fastest_lap'],ascending=True)

#### Best lap per driver compared with the best lap of the session in dry conditions

In [ ]:
obtainchart("name_acronym","delta",jointables.sort_values(by=['fastest_lap'],ascending=True))

In [ ]:
drivers_list = list(competitiveLaps_wet['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_information_qualy(driver,competitiveLaps_wet,newdataset)
jointables = pd.merge(newdataset,drivers,on=['driver_number'])
jointables.sort_values(by=['fastest_lap'],ascending=True)

#### Best lap per driver compared with the best lap of the session in wet conditions

In [ ]:
obtainchart("name_acronym","delta",jointables.sort_values(by=['fastest_lap'],ascending=True))

In [ ]:
mergequaly_dry = pd.merge(competitiveLaps_dry,drivers,on=['driver_number'])
mergequaly_dry

In [ ]:
mergequaly_wet = pd.merge(competitiveLaps_wet,drivers,on=['driver_number'])
mergequaly_wet

In [ ]:
def obtain_difference_regard_reference(row,reference,newdataset):
    difference_sector_1 = row.duration_sector_1 - reference.duration_sector_1.iloc[0]
    difference_sector_2 = row.duration_sector_2 - reference.duration_sector_2.iloc[0]
    difference_sector_3 = row.duration_sector_3 - reference.duration_sector_3.iloc[0]
    lap_duration = row.lap_duration - reference.lap_duration.iloc[0]
    new_row = {'driver_number':row.driver_number,'lap_duration':lap_duration,'difference_sector_1':difference_sector_1 ,'difference_sector_2':difference_sector_2,'difference_sector_3':difference_sector_3,'name_acronym':row.name_acronym   }
    
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
# Function done to obtain more information about the qualyfing session
def obtainInfoAboutQualySession(dataset,fecha):
    sessiondataset =dataset.query(fecha).sort_values(by='lap_duration')
    isFastestLap = []
    for index,row in sessiondataset.iterrows():
        driver = row.driver_number
        fastest_lap = sessiondataset.query("driver_number == @driver").lap_duration.min()
        if row.lap_duration == fastest_lap:
            isFastestLap.append(True)
        else:
            isFastestLap.append(False)
    sessiondataset['isFastestLap'] = isFastestLap
    return sessiondataset

In [ ]:
# In order to know when each session finished, race control dataset will be consulted.
maximumDateQ1 = "date_start <'2024-04-19T07:54:00'"
maximumDateQ2 = "date_start <'2024-04-19T08:11:00' and date_start >='2024-04-19T07:54:00'"
maximumDateQ3 = "date_start >'2024-04-19T08:11:00'"

#### Sprint Qualyfing 1

In [ ]:
q1Data = obtainInfoAboutQualySession(mergequaly_dry,maximumDateQ1).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q1Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P15 = q1Data[14:15]
P15

In [ ]:
print(
"Driver:",P15.full_name.to_string(index=False),
"Sector 1: ",P15.duration_sector_1.to_string(index=False),
"Sector 2: ",P15.duration_sector_2.to_string(index=False),
"Sector 3: ",P15.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q1Data[15::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P15,newdataset2)

newdataset2

##### Best sector times per driver

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

#### Sprint Qualyfing 2

In [ ]:
q2Data = obtainInfoAboutQualySession(mergequaly_dry,maximumDateQ2).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q2Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P10 = q2Data[9:10]
print(
"Driver:",P10.full_name.to_string(index=False),
"Sector 1: ",P10.duration_sector_1.to_string(index=False),
"Sector 2: ",P10.duration_sector_2.to_string(index=False),
"Sector 3: ",P10.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q2Data[10::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P10,newdataset2)

newdataset2

##### Best sector times per driver

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

#### Sprint Qualyfing 3

In [ ]:
q3Data = obtainInfoAboutQualySession(mergequaly_wet,maximumDateQ3).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q3Data

In [ ]:
#Reference
P1 = q3Data[:1]
print(
"Driver:",P1.full_name.to_string(index=False),
"Sector 1: ",P1.duration_sector_1.to_string(index=False),
"Sector 2: ",P1.duration_sector_2.to_string(index=False),
"Sector 3: ",P1.duration_sector_3.to_string(index=False)
)

##### Comparaison with poleman
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q3Data[1::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P1,newdataset2)
newdataset2

#### Best sector per driver

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly_dry.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly_dry.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly_dry.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

## Sprint

In [ ]:
session_key = 'session_key=9672'
url = 'https://api.openf1.org/v1/laps?'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
sprint = pd.DataFrame(data)
sprint

In [ ]:
#Stints
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

In [ ]:
stintsDataFrame = pd.DataFrame()
for index,row in stintInformation.iterrows():
    number_driver = row.driver_number
    acronym_driver = drivers.query('driver_number == @number_driver').name_acronym.to_string(index=False)
    full_name = drivers.query('driver_number == @number_driver').full_name.to_string(index=False)
    team_name = drivers.query('driver_number == @number_driver').team_name.to_string(index=False)
    if row.lap_start != row.lap_end:
        contador = row.lap_start
        while contador <=row.lap_end :
            new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':contador,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
            stintsDataFrame = pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)
            contador+=1
    else:
    
        new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':row.lap_start,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
        stintsDataFrame =pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
jointables = pd.merge(sprint,stintsDataFrame,on=['lap_number','driver_number'])
jointables

In [ ]:
obtain_data_tyres(jointables,"MEDIUM",False)

In [ ]:
#obtain_data_tyres(jointables,"SOFT",False)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 110").dropna().groupby("team_name")['lap_duration'].mean().sort_values(ascending=True))
race_pace

In [ ]:
def obtain_race_pace(dataset,array):
    fastest_lap = dataset.lap_duration.min()
    for row in dataset.iterrows():
        lap = row[1][0]
        delta = lap-fastest_lap
        array.append(delta)
    return array
arr= obtain_race_pace(race_pace,[])
race_pace.insert(1,'delta',arr)

#### Race pace per teams

In [ ]:
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 110").dropna().groupby("team_name")['duration_sector_1'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 110").dropna().groupby("team_name")['duration_sector_2'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 110").dropna().groupby("team_name")['duration_sector_3'].mean().sort_values(ascending=True))
race_pace

#### Red Bull Racing

In [ ]:
stintInformation.query('driver_number == 1 or driver_number == 11')

In [ ]:
getinfolonglaps(jointables,1,'Red Bull Racing')

In [ ]:
getinfolonglaps(jointables,11,'Red Bull Racing')

#### Ferrari

In [ ]:
stintInformation.query('driver_number == 16 or driver_number == 55')

In [ ]:
getinfolonglaps(jointables,16,'Ferrari')

In [ ]:
getinfolonglaps(jointables,55,'Ferrari')

#### Mercedes

In [ ]:
stintInformation.query('driver_number == 63 or driver_number == 44')

In [ ]:
getinfolonglaps(jointables,63,'Mercedes')

In [ ]:
getinfolonglaps(jointables,44,'Mercedes')

#### Aston Martin

In [ ]:
stintInformation.query('driver_number == 14 or driver_number == 18')

In [ ]:
getinfolonglaps(jointables,14,'Aston Martin')

In [ ]:
getinfolonglaps(jointables,18,'Aston Martin')

#### McLaren

In [ ]:
stintInformation.query('driver_number == 4 or driver_number == 81')

In [ ]:
getinfolonglaps(jointables,81,'McLaren')

In [ ]:
getinfolonglaps(jointables,4,'McLaren')

#### RB

In [ ]:
stintInformation.query('driver_number == 3 or driver_number == 22')

In [ ]:
getinfolonglaps(jointables,3,'RB')

In [ ]:
getinfolonglaps(jointables,22,'RB')

#### Williams

In [ ]:
stintInformation.query('driver_number == 2 or driver_number == 23')

In [ ]:
getinfolonglaps(jointables,2,'Williams')

In [ ]:
getinfolonglaps(jointables,23,'Williams')

#### Kick Sauber

In [ ]:
stintInformation.query('driver_number == 24 or driver_number == 77')

In [ ]:
getinfolonglaps(jointables,24,'Kick Sauber')

In [ ]:
getinfolonglaps(jointables,77,'Kick Sauber')

#### Alpine

In [ ]:
stintInformation.query('driver_number == 10 or driver_number == 31')

In [ ]:
getinfolonglaps(jointables,10,'Alpine')

In [ ]:
getinfolonglaps(jointables,31,'Alpine')

#### Haas

In [ ]:
stintInformation.query('driver_number == 20 or driver_number == 27')

In [ ]:
getinfolonglaps(jointables,20,'Haas')

In [ ]:
getinfolonglaps(jointables,27,'Haas')

## Qualyfing

### Set up
First of all, it is neccesary to obtain the data about the qualyfing
#### Race control
This section has been added in order to know which laps has been deleted and knowing what happened on track during this session as well.

In [ ]:
url = 'https://api.openf1.org/v1/race_control?'
session_key = 'session_key=9664'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race_control = pd.DataFrame(data)
race_control

### Laps

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
#session_key = 'session_key=9492'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
qualyfing = pd.DataFrame(data)
qualyfing

Lap times with track limits have to be deleted. So that, I searched inrace control dataset those laptimes not valids to be deleted from qualfing dataset. 

In [ ]:
qualyfing = qualyfing.drop(40)
qualyfing = qualyfing.drop(49)
qualyfing = qualyfing.drop(50)

As the qualyfing session has been without weather changes, I will take the fastest lap as a reference to obtain all the competitive laps.

In [ ]:
bestlap = qualyfing.loc[qualyfing.groupby(['driver_number'])['lap_duration'].idxmin()].sort_values(by=['lap_duration'],ascending=True)
bestlap[0:1]

In this case, the fastest lap is 93.66 seconds (1.33.66= so that to obtain the competitve laps the fastest lap will be multiplied by 1.07 (100.2162 seconds) due to, according to the rules all the drivers have to do unless one lap within this gap.

In [ ]:
competitiveLaps = qualyfing.query("is_pit_out_lap == False and lap_duration < 100.2162")
competitiveLaps

In [ ]:
drivers_list = list(competitiveLaps['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_information_qualy(driver,competitiveLaps,newdataset)
jointables = pd.merge(newdataset,drivers,on=['driver_number'])
jointables.sort_values(by=['fastest_lap'],ascending=True)

#### Best lap per driver compared with the best lap of the session

In [ ]:
obtainchart("name_acronym","delta",jointables.sort_values(by=['fastest_lap'],ascending=True))

#### Speed trap pace per driver

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

#### Speed trap pace per team

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("team_name","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("team_name","i2_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("team_name","st_speed",top_speed)

In [ ]:
mergequaly = pd.merge(competitiveLaps,drivers,on=['driver_number'])
mergequaly

In [ ]:
# In order to know when each session finished, race control dataset will be consulted.
maximumDateQ1 = "date_start <'2024-04-20T07:25:00'"
maximumDateQ2 = "date_start <'2024-04-20T08:00:00' and date_start >'2024-04-20T07:25:00'"
maximumDateQ3 = "date_start >'2024-04-20T08:00:00'"

### Qualyfing 1

In [ ]:
q1Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ1).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q1Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P15 = q1Data[14:15]
P15

In [ ]:
print(
"Driver:",P15.full_name.to_string(index=False),
"Sector 1: ",P15.duration_sector_1.to_string(index=False),
"Sector 2: ",P15.duration_sector_2.to_string(index=False),
"Sector 3: ",P15.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q1Data[15::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P15,newdataset2)

newdataset2

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

In [ ]:
q2Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ2).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q2Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P10 = q2Data[9:10]
print(
"Driver:",P10.full_name.to_string(index=False),
"Sector 1: ",P10.duration_sector_1.to_string(index=False),
"Sector 2: ",P10.duration_sector_2.to_string(index=False),
"Sector 3: ",P10.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q2Data[10::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P10,newdataset2)

newdataset2

#### Best sector per driver

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

#### Qualyfing 3

In [ ]:
q3Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ3).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q3Data

##### Comparaison with poleman
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P1 = q3Data[:1]
print(
"Driver:",P1.full_name.to_string(index=False),
"Sector 1: ",P1.duration_sector_1.to_string(index=False),
"Sector 2: ",P1.duration_sector_2.to_string(index=False),
"Sector 3: ",P1.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q3Data[1::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P1,newdataset2)
newdataset2

#### Best sector per driver

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

## Race

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
session_key = 'session_key=9673'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race = pd.DataFrame(data)
race

### Stints

In [ ]:
#Stints
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

In [ ]:
stintsDataFrame = pd.DataFrame()
for index,row in stintInformation.iterrows():
    number_driver = row.driver_number
    acronym_driver = drivers.query('driver_number == @number_driver').name_acronym.to_string(index=False)
    full_name = drivers.query('driver_number == @number_driver').full_name.to_string(index=False)
    team_name = drivers.query('driver_number == @number_driver').team_name.to_string(index=False)
    if row.lap_start != row.lap_end:
        contador = row.lap_start
        while contador <= row.lap_end :
            new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':contador,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
            #stintsDataFrame=stintsDataFrame.append(new_row, ignore_index=True)
            stintsDataFrame = pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)
            contador+=1
    else:
    
        new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':row.lap_start,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
        #stintsDataFrame= stintsDataFrame.append(new_row, ignore_index=True)
        stintsDataFrame =pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
stintsDataFrame

### Drivers

In [ ]:
drivers

In [ ]:
jointables = pd.merge(race,stintsDataFrame,on=['lap_number','driver_number'])
jointables

### Obtain data tyres

In [ ]:
obtain_data_tyres(jointables,'MEDIUM',True)

In [ ]:
obtain_data_tyres(jointables,'SOFT',True)

In [ ]:
obtain_data_tyres(jointables,'HARD',True)

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 105").dropna().groupby("team_name")['lap_duration'].mean().sort_values(ascending=True))
race_pace

In [ ]:
def obtain_race_pace(dataset,array):
    fastest_lap = dataset.lap_duration.min()
    for row in dataset.iterrows():
        lap = row[1][0]
        delta = lap-fastest_lap
        array.append(delta)
    return array
arr= obtain_race_pace(race_pace,[])
race_pace.insert(1,'delta',arr)

In [ ]:
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 105").dropna().groupby("team_name")['duration_sector_1'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 105").dropna().groupby("team_name")['duration_sector_2'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 105").dropna().groupby("team_name")['duration_sector_3'].mean().sort_values(ascending=True))
race_pace

### Comparaison beetween drivers
I created this section to know the drivers racepace to compare beetween team mates who were the fastest. In general, we can see that the difference is among three or four tenths except in Aston Martin case where the difference beetween them is in 9 tenths.

#### Red Bull Racing

In [ ]:
race.query("driver_number== 1 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 11 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Ferrari

In [ ]:
race.query("driver_number== 16 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 55 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Mercedes

In [ ]:
race.query("driver_number== 44 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 63 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### McLaren

In [ ]:
race.query("driver_number== 4 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 81 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Aston Martin

In [ ]:
race.query("driver_number== 14 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 18 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

##### RB

In [ ]:
race.query("driver_number== 22 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 3 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Haas

In [ ]:
race.query("driver_number== 20 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 27 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Williams

In [ ]:
race.query("driver_number== 2 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 23 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Kick Sauber

In [ ]:
race.query("driver_number== 77 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 24 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Alpine

In [ ]:
race.query("driver_number== 31 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

In [ ]:
race.query("driver_number== 10 and lap_duration <=105 and lap_duration > 94").lap_duration.mean()

#### Degradation

In [ ]:
pitstoplist = list(stintInformation.query("driver_number == 55").lap_start.values)
pitstoplist
dict_stint = {}
for f in list(stintInformation.query("driver_number == 55 ").values):
    # Stint number
    print(f[2])
    dict_stint[f[2]] = []
    #Lap end
    print(f[5])
    # Compound
    print(f[6])
lapsXDriver = jointables.query("driver_number==55 ")
mean = 0
last_lap = 0
contador = 0
total = 0
stint_number = 1
for  index,row in lapsXDriver.iterrows():
    # Valid laps
    if row.lap_number > 1:
        if row.lap_number in pitstoplist:
            stint_number+=1
        if row.lap_duration < 108:
            dict_stint[stint_number].append(row.lap_duration)
total = 0
for key in dict_stint:
    value = dict_stint[key]
    if len(np.diff(value)) != 0:
        print(np.diff(value))
        valor = np.mean(np.diff(value))
        print(valor)
        total += valor
   
print("Mean degradation",total)

    

### Pits

In [ ]:

url = 'https://api.openf1.org/v1/pit?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
pits = pd.DataFrame(data)
pits

In [ ]:
jointables = pd.merge(drivers,pits,on=['driver_number']).query("pit_duration<180")
jointables
pit_duration = pd.DataFrame(jointables.groupby('team_name')['pit_duration'].mean().sort_values(ascending=True))
pit_duration